<a href="https://colab.research.google.com/github/sunilgandipadala/AI_DL_ASSIGNMENT-PARKZAP/blob/main/NLP_based_ML_Model_Neurance_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import the necessary libraries

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [4]:
# NLP
from bs4 import BeautifulSoup
import string
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#need tokenizer
#Lemmatizer
#stemming
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
#need all the tokenizers... from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

In [5]:
#spacy module we are going to use for preprocessing
nlp = spacy.load('en_core_web_sm')
stopwords = list(STOP_WORDS)
punct = string.punctuation
punct=punct+"\n"

# READ DATA

In [40]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")


In [41]:
train_data.describe(include='all')

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
count,32165.000000,32165,32165,32165,32165.000000,32165,32165.000000,32165.000000
unique,NaN,2220,636,30121,NaN,3537,NaN,NaN
top,NaN,Levonorgestrel,Birth Control,"""Good""",NaN,18-Apr-17,NaN,NaN
freq,NaN,724,5795,8,NaN,37,NaN,NaN
mean,115487.511052,NaN,NaN,NaN,6.998912,NaN,27.893207,6.664261
std,67146.156602,NaN,NaN,NaN,3.274781,NaN,35.819035,1.588699
min,9.000000,NaN,NaN,NaN,1.000000,NaN,0.000000,0.899076
25%,56922.000000,NaN,NaN,NaN,5.000000,NaN,6.000000,6.109307
50%,115776.000000,NaN,NaN,NaN,8.000000,NaN,16.000000,6.788245
75%,173562.000000,NaN,NaN,NaN,10.000000,NaN,36.000000,7.900790


In [42]:
train_data.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [43]:
test_data.head()

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2


### Review_by_patient looks, it needs to be cleaned

In [44]:
def punctuation_removal(text):
    clean_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

train_data['review_by_patient'] = train_data['review_by_patient'].apply(punctuation_removal)
test_data['review_by_patient'] = test_data['review_by_patient'].apply(punctuation_removal)

In [45]:
def stopwords_removal(text):
    doc = nlp(text)
    return [word.text.lower() for word in doc if word.text.lower() not in stopwords ]

train_data['review_by_patient'] = train_data['review_by_patient'].apply(stopwords_removal)
test_data['review_by_patient'] = test_data['review_by_patient'].apply(stopwords_removal)

In [46]:
#Dropping numbers
import re

def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ' '.join(list_text_new)

train_data['review_by_patient'] = train_data['review_by_patient'].apply(drop_numbers)
test_data['review_by_patient'] = test_data['review_by_patient'].apply(drop_numbers)

## We have to find the sentiment of the review of the patient which will help us to predict the results

In [48]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [49]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Create list (cast to array) of compound polarity sentiment scores for reviews
train_sentiments = []
test_sentiments = []

for i in train_data.review_by_patient:
    train_sentiments.append(sid.polarity_scores(i).get('compound'))
    
for i in test_data.review_by_patient:
    test_sentiments.append(sid.polarity_scores(i).get('compound'))   
    
#sentiments...
train_sentiments = np.asarray(train_sentiments)
test_sentiments  = np.asarray(test_sentiments)

## add the sentiments to the datasets

In [50]:
train_data['sentiment'] = pd.Series(data=train_sentiments)
test_data['sentiment'] = pd.Series(data=test_sentiments)

In [51]:
# Sort train dataframe from most to least useful
df = train_data.sort_values(by='number_of_times_prescribed', ascending=False)
df.iloc[:10]

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,sentiment
404,62757,Citalopram,Depression,responded week effects lethargy bowel problems...,8,25-Mar-08,771,6.948738,-0.9246
31629,89689,Celexa,Depression,taking celexa months life completely different...,9,29-Jan-10,559,6.260555,-0.2037
10692,77107,Lorcaserin,Weight Loss,update year experience belviq dropped pounds y...,10,21-Sep-14,556,8.755092,0.7269
23744,126310,Viibryd,Depression,atypical depression tried wellbutrin allergic ...,9,7-Sep-11,513,8.400719,-0.8578
13466,182492,Mirena,Birth Control,love mirena recommended friends m kids wanted ...,10,1-Oct-08,505,8.412500,0.7906
28671,197749,Desvenlafaxine,Depression,trying antidepressants feeling exactly right ...,10,24-Jun-08,504,7.252806,0.8750
7063,138983,Phentermine,Weight Loss,lot people forum extremely unrealistic drug me...,8,18-Sep-09,476,6.494641,0.4939
16203,222844,Topamax,Obesity,topamax little month lost pounds doctor gave w...,10,24-May-13,467,8.122917,-0.7024
22884,29632,Topiramate,Obesity,topamax little month lost pounds doctor gave w...,10,24-May-13,467,8.122917,-0.7024
3943,151191,OxyContin,Pain,think tried pain medication known man years ...,10,12-Feb-10,463,8.024776,-0.5106


In [52]:
# ################################


min_rating = train_data.effectiveness_rating.min()
max_rating = test_data.effectiveness_rating.max()

def scale_rating(rating):
    rating -= min_rating
    rating = rating/(max_rating -1)
    rating *= 5
    rating = int(round(rating,0))
    
    if(int(rating) == 0 or int(rating)==1 or int(rating)==2):
        return 0
    else:
        return 1


    
train_data['new_eff_score'] = train_data.effectiveness_rating.apply(scale_rating)
test_data['new_eff_score'] = test_data.effectiveness_rating.apply(scale_rating)

# X.drop("effectiveness_rating",axis=1,inplace=True)

In [53]:
big_df = pd.concat([df.drop("base_score",axis=1), test_data], ignore_index=True)

In [54]:
# Make dictionary of use_case_of_drug, each value will be a dataframe of all of the drugs used to treat the given use_case
help_dict = {}

# Iterate over conditions
for i in big_df.use_case_for_drug.unique():
    
    temp_ls = []
    
    # Iterate over drugs within a given condition
    for j in big_df[big_df.use_case_for_drug == i].name_of_drug.unique():
        
        # If there are at least 0 reviews for a drug, save its name and average rating in temporary list
        if np.sum(big_df.name_of_drug == j) >= 0:
            temp_ls.append((j, np.sum(big_df[big_df.name_of_drug == j].effectiveness_rating) / np.sum(big_df.name_of_drug == j)))
        
    # Save temporary list as a dataframe as a value in help dictionary, sorted best to worst drugs
    help_dict[i] = pd.DataFrame(data=temp_ls, columns=['drug', 'average_rating']).sort_values(by='average_rating', ascending=False).reset_index(drop=True)

In [55]:
count = 0
def rank_drug_test(name):
    global count 
    use_case = test_data.iloc[count][2]
    count = count + 1
#     print(use_case)
    a = help_dict[use_case]   ## this will create a dataframe
    rank=a.index[a['drug'] == name].tolist()
    
    if(rank[0] > 30 ):                                  ## those who has less than 30 reviews are ranked zero
        return 0
    else:
        return (30-rank[0])                             ## The topn review will  get 30 as rank ( higher the better)
    
    
    
def rank_drug_train(name):
    global count 
    use_case = train_data.iloc[count][2]
    count = count + 1
#     print(use_case)
    a = help_dict[use_case]   ## this will create a dataframe
    rank=a.index[a['drug'] == name].tolist()
    
    if(rank[0] > 30 ):                                  ## those who has less than 30 reviews are ranked zero
        return 0
    else:
        return (30-rank[0])   

In [56]:
train_data.head(5)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,sentiment,new_eff_score
0,206461,Valsartan,Left Ventricular Dysfunction,effect combination bystolic mg fish oil,9,20-May-12,27,8.022969,0.0000,1
1,95260,Guanfacine,ADHD,son halfway fourth week intuniv concerned bega...,8,27-Apr-10,192,7.858458,0.9169,1
2,92703,Lybrel,Birth Control,oral contraceptive pill cycle happy light peri...,5,14-Dec-09,17,6.341969,0.7506,0
3,35696,Buprenorphine / naloxone,Opiate Dependence,suboxone completely turned life feel healthi...,9,27-Nov-16,37,6.590176,0.9403,1
4,155963,Cialis,Benign Prostatic Hyperplasia,day mg started work rock hard erections experi...,2,28-Nov-15,43,6.144782,-0.6249,0


In [58]:
train_data['rank_of_drug'] = train_data['name_of_drug'].apply(rank_drug_train)
count =0
#test_data['rank_of_drug'] = test_data['name_of_drug'].apply(rank_drug_test)

KeyError: ignored

In [59]:
train_data.drop(["patient_id","name_of_drug",
        "use_case_for_drug",
        "review_by_patient","drug_approved_by_UIC","rank_of_drug"],axis=1,inplace=True)


test_data.drop(["patient_id","name_of_drug",
        "use_case_for_drug",
        "review_by_patient","drug_approved_by_UIC"],axis=1,inplace=True)

In [62]:
train_data.drop("sentiment",axis=1,inplace=True)

In [63]:
test_data.drop("sentiment",axis=1,inplace=True)

In [64]:
train_data.head(3)

,effectiveness_rating,number_of_times_prescribed,base_score,new_eff_score
0,9,27,8.022969,1
1,8,192,7.858458,1
2,5,17,6.341969,0


In [81]:
X = train_data.drop("base_score",axis=1)
y= train_data.base_score

from sklearn.preprocessing import  MinMaxScaler
sc= MinMaxScaler()
X= sc.fit_transform(X)
y= y.values.reshape(-1,1)


test_data = sc.transform(test_data)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)



#Spliting into X and Y ( 30% testing - 70 training )

In [67]:
from sklearn.model_selection import train_test_split as split

X_train, X_test, y_train, y_test = split(X,y, test_size=0.3,random_state=51)

In [68]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

In [69]:
# XGB parameters
xgb_reg_params = {
#     'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.15, 0.25, 0.01)),
    'n_estimators':     hp.choice('n_estimators', np.arange(1000,1800,10, dtype=int)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 15, 1, dtype=int)),
#     'gamma': hp.choice('gamma', np.arange(0, 0.4, 0.1, dtype=int)),
#     'subsample':        hp.uniform('subsample', 0.8, 1),
}


xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}


from sklearn.metrics import mean_squared_error
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y,pred: np.sqrt(mean_squared_error(y, pred))

In [70]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [71]:
import lightgbm as lgb

In [72]:
class HPOpt(object):
    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train#.ravel()
        self.y_test  = y_test#.ravel()

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def lgb_reg(self, para):
        reg = lgb.LGBMRegressor(**para['reg_params'])
        return self.train_reg(reg, para)


    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [73]:
obj = HPOpt(X_train, X_test, y_train, y_test)

In [74]:
xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
print (xgb_opt)

[17:07:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:07:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [75]:
model = xgb.XGBRegressor(
                         learning_rate=0.17,
                         n_estimators =1400,
                         max_depth =5
                        )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


from sklearn import metrics

print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

[17:23:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.06021341015194316


In [76]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)   # best = 1

0.9985765926182271

#predicting

In [82]:
test_data

array([[2.33333333, 0.01297017, 1.        ],
       [3.77777778, 0.01167315, 1.        ],
       [1.33333333, 0.00518807, 0.        ],
       ...,
       [4.66666667, 0.01297017, 1.        ],
       [0.66666667, 0.01037613, 1.        ],
       [5.        , 0.01167315, 1.        ]])

In [83]:
result = model.predict(test_data)

In [85]:
temp_df=pd.read_csv("test.csv")

result_data=temp_df['patient_id']

result_data = pd.DataFrame(result_data)

result_data['base_score']=result

#writing into OUTPUT FILE..
result_data.to_csv('output.csv',index=False) 
